# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality via LangChain
- define a set of functions by reading configs from obbtools.yaml file and creating LangChain tools 
- use LangChain to generate prompt using the tools and system prompt


In [1]:
import os
import dotenv
import warnings

import yaml

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import langchain
from langchain_core.tools import StructuredTool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI

# langchain is still on pydantic v1
import pydantic.v1
from pydantic.v1 import BaseModel, Field # <-- Uses v1 namespace

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

from BB_agent_tool import BB_agent_tool, bb_agent_system_prompt

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'pydantic       {pydantic.v1.__version__}')


pandas         2.2.2
obb            4.2.1
openai         1.30.1
langchain      0.2.0
pydantic       1.10.15


# Connect to OpenBB

In [3]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)


In [4]:
obb.equity.search("Merck", provider="sec").to_df().head(3)


,symbol,name,cik
0,MRK,"Merck & Co., Inc.",310158


In [5]:
symbol = "MRK"
company = "Merck"


In [6]:
obj = obb.equity.price.quote(symbol)
obj


OBBject

id: 066536b6-4051-7dff-8000-0f6cb933c9b8
results: [{'symbol': 'MRK', 'asset_type': None, 'name': 'Merck & Co., Inc.', 'excha...
provider: fmp
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'fmp'}, 'standa...

In [7]:
obj.results


[FMPEquityQuoteData(symbol=MRK, asset_type=None, name=Merck & Co., Inc., exchange=NYSE, bid=None, bid_size=None, bid_exchange=None, ask=None, ask_size=None, ask_exchange=None, quote_conditions=None, quote_indicators=None, sales_conditions=None, sequence_number=None, market_center=None, participant_timestamp=None, trf_timestamp=None, sip_timestamp=None, last_price=129.49, last_tick=None, last_size=None, last_timestamp=2024-05-24 20:00:02+00:00, open=131.45, high=131.45, low=129.16, close=None, volume=5829244, exchange_volume=None, prev_close=131.09, change=-1.6, change_percent=-0.012204999999999999, year_high=133.1, year_low=99.14, price_avg50=127.7616, price_avg200=115.32135, avg_volume=8199988, market_cap=327973566900.0, shares_outstanding=2532810000, eps=0.9, pe=143.88, earnings_announcement=2024-07-30 12:30:00+00:00)]

# Prompt OpenAI 

In [8]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0


In [9]:
chat = ChatOpenAI(model=MODEL, temperature=0.0, streaming=True)

prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                bb_agent_system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

chain = prompt_template | chat

memory = ChatMessageHistory(llm=chat, max_token_limit=1000)

memory.add_user_message("What is the airspeed velocity of an unladen swallow")

text_response = ""
display_handle = display(HTML("<p>"), display_id=True)

for chunk in chain.stream({"messages": memory.messages}):
    text_response += chunk.content
    display_handle.update(HTML(f"<p>{text_response}</p>"))
    if hasattr(chunk, 'response_metadata'):
        response_metadata = chunk.response_metadata


# Question Answering Agent
### Create tools

In [10]:
# dict to store tools
tool_dict={}


In [11]:
# LangChain needs call signature as pydantic class

class SymbolLimitSchema(BaseModel):
    symbol: str
    limit: int

class SymbolSchema(BaseModel):
    symbol: str

class QuerySchema(BaseModel):
    query: str = Field(description="The search string to match to the stock symbol.")
    # limit not supported for provider=sec
    # limit: int = Field(description="The maximum number of values to return")

schema_dict = {
    "SymbolSchema": SymbolSchema,
    "QuerySchema": QuerySchema,
    "SymbolLimitSchema": SymbolLimitSchema,
}


In [12]:
# make a custom tool

# metadata not used for custom tool
yaml_get_10k_item1_from_symbol="""
name: get_10k_item1_from_symbol
description: Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.
openapi_path: null
parameters:
  symbol:
    description: The symbol to get the 10-K item 1 for
    type: string
example_parameter_values:
- symbol: MSFT
"""
yaml.safe_load(yaml_get_10k_item1_from_symbol)

def fn_get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except Exception as exc:
        print(exc)
    # always return a list of dicts
    return [{'item1': item1_text}]

get_10k_item1_from_symbol = StructuredTool.from_function(
    func=fn_get_10k_item1_from_symbol,
    name="get_10k_item1_from_symbol",
    description="Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    args_schema=SymbolSchema
)

tool_dict["get_10k_item1_from_symbol"]= get_10k_item1_from_symbol
tool_response = get_10k_item1_from_symbol.run("MSFT")
len((tool_response)[0]['item1'])


143226

In [13]:
def obb_function_factory(fn_metadata):
    # get obb method object based on path, e.g. /api/v1/equity/price/quote -> obb.equity.price.quote
    parts = fn_metadata["openapi_path"].removeprefix("/api/v1/").split("/")
    op = obb
    for part in parts:
        op = op.__getattribute__(part)
    singular = fn_metadata.get("singular", 0)
    default_args = fn_metadata.get("default_parameters", {})
    override_args = fn_metadata.get("override_parameters", {})

    # return a closure based on value of op etc.
    def tool_fn(**kwargs):
        """call op and return results without obb metadata"""
        # always use override args
        for k, v in override_args.items():
            kwargs[k] = v
        # use default if not already present
        for k, v in default_args.items():
            if k not in kwargs:
                kwargs[k] = v
        retobj = op(**kwargs)
        retlist = [r.model_dump_json() for r in retobj.results]
        if len(retlist):
            if singular == 1:
                # return first
                return retlist[0]
            elif singular == -1:
                # return last
                return retlist[-1]
        # no retlist or not singular
        return retlist

    return tool_fn


In [14]:
# Load obb tool defs from YAML file
with open('obbtools.yaml', 'r') as file:
    fn_yaml_list = yaml.safe_load(file)

for fn_metadata in fn_yaml_list:

    # instantiate tool and add to tool_dict
    tool_dict[fn_metadata["name"]] = StructuredTool.from_function(
        func=obb_function_factory(fn_metadata),
        name=fn_metadata["name"],
        description=fn_metadata["description"],
        args_schema=schema_dict[fn_metadata["args_schema"]]
    )

    # test tool
    print(fn_metadata["name"])
    for p in fn_metadata['example_parameter_values']:
        print(tool_dict[fn_metadata["name"]].run(tool_input=dict(**p)))
    print()


get_equity_search_symbol
['{"symbol":"AVGO","name":"Broadcom Inc.","cik":"1730168"}']

get_equity_price_quote
['{"symbol":"AAPL","asset_type":"EQUITY","name":"Apple Inc.","exchange":"NMS","bid":189.97,"bid_size":100,"bid_exchange":null,"ask":190.06,"ask_size":100,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":189.98,"last_tick":null,"last_size":null,"last_timestamp":null,"open":188.82,"high":190.58,"low":188.0404,"close":null,"volume":35941665,"exchange_volume":null,"prev_close":186.88,"change":null,"change_percent":null,"year_high":199.62,"year_low":164.08,"ma_50d":175.9172,"ma_200d":180.9828,"volume_average":63881565.0,"volume_average_10d":49758620.0,"currency":"USD"}']

get_company_profile_json
['{"symbol":"NVDA","name":"NVIDIA Corporation","cik":"0001045810","cusip":"67066G104","isin":"US67066G1040","lei":null,"

### Create agent

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=bb_agent_system_prompt,)),
     MessagesPlaceholder(variable_name='chat_history', optional=True),
     HumanMessagePromptTemplate(prompt=PromptTemplate(
         input_variables=['input'], template='{input}')),
     MessagesPlaceholder(variable_name='agent_scratchpad')]
)

tool_list = list(tool_dict.values())
agent = create_tool_calling_agent(chat, tool_list, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)


In [16]:
def escape_output(text):
    # escape $ characters to avoid markdown interpreting them as latex
    return text.replace('$', '\$', )


In [23]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

response = agent_executor.invoke({"input": "get the latest price quote for Delta Air lines"})

display(Markdown(escape_output(response["output"])))



> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Delta Air Lines'}`


['{"symbol":"DAL","name":"DELTA AIR LINES, INC.","cik":"27904"}']
Invoking: `get_equity_price_quote` with `{'symbol': 'DAL'}`


['{"symbol":"DAL","asset_type":"EQUITY","name":"Delta Air Lines, Inc.","exchange":"NYQ","bid":51.6,"bid_size":1400,"bid_exchange":null,"ask":51.61,"ask_size":800,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":51.84,"last_tick":null,"last_size":null,"last_timestamp":null,"open":51.45,"high":51.88,"low":51.11,"close":null,"volume":5191145,"exchange_volume":null,"prev_close":51.2,"change":null,"change_percent":null,"year_high":53.86,"year_low":30.6,"ma_50d":48.9044,"ma_200d":41.34215,"volume_average":9424714.0,"volume_average_10d":5586280.0,"currency":"USD"}']The latest price quo

The latest price quote for Delta Air Lines, Inc. (symbol: DAL) is \$51.84. Here are some additional details:

- **Open Price:** \$51.45
- **High Price:** \$51.88
- **Low Price:** \$51.11
- **Previous Close:** \$51.20
- **Volume:** 5,191,145
- **52-Week High:** \$53.86
- **52-Week Low:** \$30.60

If you need more information, feel free to ask!

In [18]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

with open("questions.txt") as infile:
    for str in infile:
        str = str.strip()
        if str:
            print(str)
            if str[0] != '#':
                response = agent_executor.invoke({"input": str})
                display(Markdown(escape_output(response["output"])))


# note that airlines is not in the the desc for LUV
What is the stock symbol for Delta Air Lines?


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Delta Air Lines'}`


['{"symbol":"DAL","name":"DELTA AIR LINES, INC.","cik":"27904"}']The stock symbol for Delta Air Lines is **DAL**.

> Finished chain.


The stock symbol for Delta Air Lines is **DAL**.

# note misspelling
What is the last price for Microsfot


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Microsoft'}`


['{"symbol":"MSFT","name":"MICROSOFT CORP","cik":"789019"}']
Invoking: `get_equity_price_quote` with `{'symbol': 'MSFT'}`


['{"symbol":"MSFT","asset_type":"EQUITY","name":"Microsoft Corporation","exchange":"NMS","bid":430.02,"bid_size":100,"bid_exchange":null,"ask":430.23,"ask_size":100,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":430.16,"last_tick":null,"last_size":null,"last_timestamp":null,"open":427.21,"high":431.055,"low":424.41,"close":null,"volume":11786718,"exchange_volume":null,"prev_close":427.0,"change":null,"change_percent":null,"year_high":433.6,"year_low":309.45,"ma_50d":416.2698,"ma_200d":376.96725,"volume_average":19575223.0,"volume_a

The last price for Microsoft Corporation (symbol: MSFT) is \$430.16.

Please summarize item 1 from the latest MSFT annual report


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Microsoft'}`


['{"symbol":"MSFT","name":"MICROSOFT CORP","cik":"789019"}']
Invoking: `get_10k_item1_from_symbol` with `{'symbol': 'MSFT'}`


[{'item1': "Note About Forward-Looking Statements\nThis report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are “forward-looking statements” within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: “Business” (Part I, Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K), and “Management’s Discussion and Analysis of Financial Condition and Results of Operations” (Part II, Item 7 of th

...

It looks like you're trying to concatenate multiple video files using FFmpeg. However, this query is not related to stock market information or individual stocks. 

If you need assistance with FFmpeg commands or video processing, I recommend checking out resources like the [FFmpeg documentation](https://ffmpeg.org/documentation.html) or forums like [Stack Overflow](https://stackoverflow.com/) where you can get help from the community.

If you have any questions about stocks or specific companies, feel free to ask!

Provide latest statistics on short interest for symbol MRK?


> Entering new AgentExecutor chain...
I currently do not have access to a tool that provides short interest data. However, you can typically find this information on financial news websites, stock market analysis platforms, or through your brokerage's research tools. If you need any other information about Merck & Co., Inc. (symbol: MRK), please let me know!

> Finished chain.


I currently do not have access to a tool that provides short interest data. However, you can typically find this information on financial news websites, stock market analysis platforms, or through your brokerage's research tools. If you need any other information about Merck & Co., Inc. (symbol: MRK), please let me know!

What is the stock symbol for southwest airlines?


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'southwest airlines'}`


['{"symbol":"LUV","name":"SOUTHWEST AIRLINES CO","cik":"92380"}']The stock symbol for Southwest Airlines is **LUV**.

> Finished chain.


The stock symbol for Southwest Airlines is **LUV**.